<a href="https://colab.research.google.com/github/ClauCll/Curso_ML_A_Z/blob/master/Regresi%C3%B3n/Regresion_Bajas_Antiguedad_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cuantas serán las bajas voluntarias de los clientes fijos para un periodo x. (Agosto - Diciembre 2020)
 

In [182]:
import pandas as pd
import numpy as np

#Gráfico
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt 
from matplotlib import style
import matplotlib.ticker as ticker
import seaborn as sns

import gspread
from oauth2client.client import GoogleCredentials 
from google.colab import auth
# Autoriza Claudia
auth.authenticate_user()
gc = gspread.authorize (GoogleCredentials.get_application_default())

### leemos datos

In [183]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [184]:
data = pd.read_csv('/content/drive/My Drive/Dataset/M_BAJAS.txt',delimiter='~')

In [185]:
data.head(5)

,ANIO_BAJA,MES_BAJA,FECHA_BAJA,ANTIGUEDAD,CANT
0,2019,11,01/11/2019,1068,3
1,2020,4,01/04/2020,860,1
2,2019,11,05/11/2019,1673,1
3,2020,2,01/02/2020,1419,1
4,2019,8,01/08/2019,782,1


In [186]:
encabezados= ['Año','Mes','Fecha','Antiguedad','Cant_Bajas']
data.columns=encabezados

In [187]:
data.Fecha = pd.to_datetime(data.Fecha)

In [188]:
data.head(5)

,Año,Mes,Fecha,Antiguedad,Cant_Bajas
0,2019,11,2019-01-11,1068,3
1,2020,4,2020-01-04,860,1
2,2019,11,2019-05-11,1673,1
3,2020,2,2020-01-02,1419,1
4,2019,8,2019-01-08,782,1


In [189]:
data.dtypes

Año                    int64
Mes                    int64
Fecha         datetime64[ns]
Antiguedad             int64
Cant_Bajas             int64
dtype: object

In [190]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

Año           0
Mes           0
Fecha         0
Antiguedad    0
Cant_Bajas    0
dtype: int64


In [191]:
data.describe(include='all')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


,Año,Mes,Fecha,Antiguedad,Cant_Bajas
count,17101.000000,17101.000000,17101,17101.000000,17101.000000
unique,NaN,NaN,315,NaN,NaN
top,NaN,NaN,2020-01-02 00:00:00,NaN,NaN
freq,NaN,NaN,848,NaN,NaN
first,NaN,NaN,2019-01-01 00:00:00,NaN,NaN
last,NaN,NaN,2020-12-06 00:00:00,NaN,NaN
mean,2019.358459,5.469797,NaN,902.821531,1.561312
std,0.479562,3.262741,NaN,493.846888,1.115205
min,2019.000000,1.000000,NaN,0.000000,1.000000
25%,2019.000000,3.000000,NaN,489.000000,1.000000


In [197]:
data.info 

<bound method DataFrame.info of         Año  Mes      Fecha Antiguedad  Cant_Bajas
0      2019   11 2019-01-11          6           3
1      2020    4 2020-01-04          6           1
2      2019   11 2019-05-11          7           1
3      2020    2 2020-01-02          6           1
4      2019    8 2019-01-08          6           1
...     ...  ...        ...        ...         ...
17096  2019    8 2019-01-08          6           1
17097  2019    4 2019-01-04          3           1
17098  2019   11 2019-01-11          5           1
17099  2020    7 2020-01-07          6           1
17100  2019    3 2019-03-23          5           1

[17101 rows x 5 columns]>

In [ ]:
#Agrupamos la columna antiguedad
#1 : 1 TRIMESTRE (0-93 DIAS)
#2 : 2 TRIMESTRE (94-186 DIAS)
#3 : 3 TRIMESTRE (187-279 DIAS)
#4 : 4 TRIMESTRE (280-365 DIAS)
#5 : A 2 AÑOS (366-744 DIAS) 
#6 : A 4 AÑOS (745-1488 DIAS)
#7 : Mayor 4 años (Máximo DIAS 17101) 
bins=[0,93,186,279,365,744,1488,17110]
names=['1','2','3','4','5','6','7']
data['Antiguedad'] = pd.cut(data['Antiguedad'],bins,labels=names)

In [200]:
data.head(5)

,Año,Mes,Fecha,Antiguedad,Cant_Bajas
0,2019,11,2019-01-11,6,3
1,2020,4,2020-01-04,6,1
2,2019,11,2019-05-11,7,1
3,2020,2,2020-01-02,6,1
4,2019,8,2019-01-08,6,1


In [201]:
 #validamos datos faltantes
 print(pd.isnull(data).sum())

Año           0
Mes           0
Fecha         0
Antiguedad    1
Cant_Bajas    0
dtype: int64


In [202]:
data.dropna(subset = ['Antiguedad'] , axis=0, inplace=True)

In [203]:
grupo = data.groupby(by = ['Año','Mes','Antiguedad']).sum()

In [205]:
grupo= pd.DataFrame(grupo).reset_index()

In [206]:
grupo

,Año,Mes,Antiguedad,Cant_Bajas
0,2019,1,1,29.0
1,2019,1,2,28.0
2,2019,1,3,35.0
3,2019,1,4,77.0
4,2019,1,5,656.0
...,...,...,...,...
163,2020,12,3,NaN
164,2020,12,4,NaN
165,2020,12,5,NaN
166,2020,12,6,NaN


In [207]:
grupo.corr(method ='pearson')

,Año,Mes,Cant_Bajas
Año,1.000000,0.000000,0.000343
Mes,0.000000,1.000000,-0.029277
Cant_Bajas,0.000343,-0.029277,1.000000


In [20]:
x= np.array(data.drop(['Cant_Bajas'],ax1))
y= np.array(data['Cant_Bajas'])

In [21]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [28]:
from sklearn.svm import SVR

svr_ = SVR(kernel='rbf',C=1.0,epsilon=0.2)
svr_.fit(x_train, y_train)
y_pred= svr_.predict(x_test)

In [29]:
svr_.score(x_train,y_train)

-0.102827173078758

In [37]:
from sklearn.preprocessing import PolynomialFeatures
poly_reg = PolynomialFeatures(degree = 3)
x_poly = poly_reg.fit_transform(x_train)

In [ ]:
print(pd.DataFrame(x_poly))

In [39]:
from sklearn.linear_model import LinearRegression
lineal_poly = LinearRegression()
lineal_poly.fit(x_poly,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [40]:
lineal_poly.score(x_poly,y_train)

0.06117165339805852